In [ ]:
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

from torch import nn
from torch import optim

import torch
import torch.nn.functional as F

In [ ]:
# !unzip archive.zip -d data

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = datasets.ImageFolder('data/hotdog/train', transform=train_transform)
test_dataset = datasets.ImageFolder('data/hotdog/test', transform=test_transform)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
net = models.resnet18(pretrained=True)

# freeze parameters in all layers
for param in net.parameters():
    param.requires_grad = False

# switch the final layer (named "fc") to a fully-connected with 2 outputs
# note: parameters of newly created modules have unfrozen parameters by default
num_features = net.fc.in_features
net.fc = nn.Linear(num_features, 2)

net = net.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
net.train()  # set network to training phase
    
epochs = 25

# for each pass of the training dataset
for epoch in range(epochs):
    train_loss, train_correct, train_total = 0, 0, 0
    
    # for each batch of training examples
    for batch_index, (inputs, labels) in enumerate(train_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  # zero the parameter gradients
        outputs = net(inputs)  # forward pass
        loss = criterion(outputs, labels)  # compare output with ground truth
        loss.backward()  # backpropagation
        optimizer.step()  # update network weights

        # record statistics
        _, preds = torch.max(outputs.data, 1)
        train_loss += loss.item()
        train_correct += (preds == labels).sum().item()
        train_total += len(labels)
        
        # print statistics every 100 batches
        if (batch_index + 1) % 1 == 0:
            print(f'Epoch {epoch + 1}, ' +
                  f'Batch {batch_index + 1}, ' +
                  f'Train Loss: {(train_loss/1):.5f}, ' +
                  f'Train Accuracy: {(train_correct/train_total):.5f}')
            
            train_loss, train_correct, train_total = 0, 0, 0

In [ ]:
net.eval()  # set network to evaluation phase

test_loss = 0
test_correct = 0
test_total = len(test_dataloader.dataset)

with torch.no_grad():  # detach gradients so network runs faster
    
    # for each batch of testing examples
    for batch_index, (inputs, labels) in enumerate(test_dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)  # forward pass
        
        # record loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # select largest output as prediction
        _, preds = torch.max(outputs.data, 1)
        
         # compare prediction with ground truth and mark as correct if equal
        test_correct += (preds == labels).sum().item()

print(f'Test Loss: {(test_loss/len(test_dataloader)):.5f}, ' +
      f'Test Accuracy: {(test_correct/test_total):.5f} ' +
      f'({test_correct}/{test_total})')

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

test_dataloader_iterator = iter(test_dataloader)
inputs, _ = test_dataloader_iterator.next()

images = make_grid(inputs[:6], nrow=2)
images = images.numpy().transpose((1, 2, 0))
images = np.clip(std * images + mean, 0, 1)
plt.figure(figsize = (8, 12))
plt.axis('off')
plt.imshow(images)

In [ ]:
inputs = inputs.to(device)
outputs = net(inputs)

# select largest output as prediction
_, preds = torch.max(outputs.data, 1)

labels_dict = {0: 'Hot Dog', 1: 'Not Hot Dog'}
pred_labels = [labels_dict[pred] for pred in preds[:6].cpu().numpy()]

for i, pred_label in enumerate(pred_labels, 1):
    print(f'{i}. {pred_label}')